# Connect from PHP

In [2]:
import pandas as pd

# Get variable from PHP
param_file_path = 'C:/xampp/htdocs/DEMO/temp/player_id.txt'
with open(param_file_path, 'r') as file:
    player_id_str = file.read().strip()

# Convert to integer
player_id = int(player_id_str)

print(f"Processing report for player ID: {player_id}")

Processing report for player ID: 969


# Connect to MySQL

In [4]:
df = pd.read_excel('data/fake_game_data.xlsx')
df_playerData = pd.read_excel('data/fake_player_data.xlsx')

# Data Preprocess

## Convert format

In [7]:
import json
import numpy as np

# Data preprocess
df = df.dropna()  # Drop no-values
df = df.drop_duplicates()

# Get mean of json array
def process_json_array(json_str):
    try:
        # Convert string into json array
        array = json.loads(json_str)
        # Filter 0
        filtered_array = [x for x in array if x != 0]
        # If array not null, return value
        return np.mean(filtered_array) if filtered_array else np.nan
    except json.JSONDecodeError:
        return np.nan

# Apply the function
df['remainHP'] = df['remainHP'].apply(process_json_array)
df['remainBullet'] = df['remainBullet'].apply(process_json_array)

In [8]:
print(df)

    id  playerId  killNo  deathNo  surviveTime  collisionNo  bulletCollision  \
0    0        60      49       46        18.42           51               21   
1    1        61      21       22       982.04            3               52   
2    2       619      75       65       668.94            0               71   
3    3       565      75        4       852.66           44               88   
4    4       969      99       77       531.44            0               38   
..  ..       ...     ...      ...          ...          ...              ...   
95  95        71      96       47       502.95           41               98   
96  96       974       7       20       102.34           79               22   
97  97        67      27       15       501.87           39               97   
98  98       415      71       82       435.48           43               81   
99  99       976      79       61       786.51           96               52   

    bulletCollisionOnLiving  remainHP  

## Adjust values with durationOfRound < 30 mins

In [10]:
# Set standard time (30 minutes = 1800 seconds)
standard_time_seconds = 1800

# Specify the column names to exclude
columns_to_exclude = ['playerId', 'durationOfRound', 'playTime']
columns_not_to_round = ['surviveTime', 'totalVoiceDetectionDuration', 'outfitsTime', 'remainBullet', 'remainHP']
# Get columns to process by excluding the specified columns
columns_to_process = [col for col in df.columns if col not in columns_to_exclude]

# Create a new DataFrame to store the results
df_adjusted = pd.DataFrame()

# Retain the playerId, durationOfRound, playTime
df_adjusted['playerId'] = df['playerId']
df_adjusted['durationOfRound'] = df['durationOfRound']
df_adjusted['playTime'] = df['playTime']

# Define a function to adjust each column based on durationOfRound
def adjust_column(df, column):
    # Make sure durationOfRound is int
    df['durationOfRound'] = pd.to_numeric(df['durationOfRound'], errors='coerce')

    # Create a new column to store the adjusted data
    adjusted_column_name = f'{column}'
    df_adjusted[adjusted_column_name] = df[column]
    
    # Calculate the adjustment factor for rows where durationOfRound is less than standard time
    mask = df['durationOfRound'] < standard_time_seconds
    adjustment_factor = standard_time_seconds / df.loc[mask, 'durationOfRound']
    
    # Apply the adjustment factor to the selected rows
    df_adjusted.loc[mask, adjusted_column_name] = df.loc[mask, column] * adjustment_factor
    
    # Convert adjusted values to integers
    if column not in columns_not_to_round:
        df_adjusted[adjusted_column_name] = df_adjusted[adjusted_column_name].round().astype('int64')
    
# Iterate through all columns to process (excluding the first column)
for column in columns_to_process:
    adjust_column(df, column)

# Print the final DataFrame
print(df_adjusted)

    playerId  durationOfRound                playTime   id  killNo  deathNo  \
0         60           555.21 2023-10-12 08:05:48.705    0     159      149   
1         61            63.83 2023-09-23 08:05:48.705   28     592      620   
2        619           177.83 2024-06-30 08:05:48.705   20     759      658   
3        565           327.48 2023-11-27 08:05:48.705   16     412       22   
4        969           679.29 2023-09-19 08:05:48.705   11     262      204   
..       ...              ...                     ...  ...     ...      ...   
95        71           384.90 2024-07-29 08:05:48.708  444     449      220   
96       974           310.86 2024-05-06 08:05:48.708  556      41      116   
97        67           953.69 2023-11-22 08:05:48.708  183      51       28   
98       415           947.34 2024-03-21 08:05:48.708  186     135      156   
99       976           832.27 2024-07-19 08:05:48.709  214     171      132   

     surviveTime  collisionNo  bulletCollision  bul

C:\Users\hohyi\AppData\Local\Temp\ipykernel_26920\809998406.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[   0.           28.199906     20.24405331   16.48955661   10.59930221
   12.07664645   12.6438531    14.94626462   44.39648528   18.80943258
   22.79462047   21.48554067  108.91488503   25.40358038   40.1075902
   48.39750484  459.33014354   64.41834028   44.62932863   43.86246168
   96.70660291  292.63760935   64.97875064   77.87663889  392.9058663
   58.43169335  110.82431504  545.51577057  268.57081957  137.12664513
   54.79507656 4677.28415759   96.22452389   66.67714343  168.47437097
  483.35123523  459.18367347  194.27104603   82.23328284  114.27083164
   89.20275042  101.27902509  158.19871097  106.991789    109.20824026
   89.30737172  119.36339523   92.54802433  145.03458     143.37500203
   99.6600485   123.41861497  143.97120576  115.95823559  188.13510113
  148.9214477   103.55880661  218.32

## Categorize adjusted data

In [12]:
# Exclude the first, second and third column (playerId, durationOfRound, playTime)
adjusted_columns_to_process = df_adjusted.columns[3:]

# Create a new DataFrame to store the results
df_categorized = pd.DataFrame()

# Retain the playerId, playTime column
df_categorized['playerId'] = df_adjusted['playerId']
df_categorized['playTime'] = df_adjusted['playTime']

# Convert columns into numeric
df_adjusted[adjusted_columns_to_process] = df_adjusted[adjusted_columns_to_process].apply(pd.to_numeric, errors='coerce')
# Fill NaN with 0
df_adjusted[adjusted_columns_to_process] = df_adjusted[adjusted_columns_to_process].fillna(0)

# Define a function to categorize each column based on standard deviation
def process_column(df, column):
    # Calculate quantiles
    quantiles = df[column].quantile([0, 0.1, 0.3, 0.7, 0.9, 1]).values
    def categorize_value(value):
        if value <= quantiles[1]:
            return '非常低'
        elif quantiles[1] < value <= quantiles[2]:
            return '低'
        elif quantiles[2] < value <= quantiles[3]:
            return '中'
        elif quantiles[3] < value <= quantiles[4]:
            return '高'
        else:
            return '非常高'

    # Apply the categorization and create a new column to store the results
    df_categorized[column] = df_adjusted[column].apply(categorize_value)

# Iterate through all columns to process (excluding the first column)
for column in adjusted_columns_to_process:
    process_column(df_adjusted, column)

print(df_categorized)

    playerId                playTime   id killNo deathNo surviveTime  \
0         60 2023-10-12 08:05:48.705  非常低      中       中         非常低   
1         61 2023-09-23 08:05:48.705    低      高       高         非常高   
2        619 2024-06-30 08:05:48.705  非常低    非常高     非常高         非常高   
3        565 2023-11-27 08:05:48.705  非常低      高     非常低           高   
4        969 2023-09-19 08:05:48.705  非常低      中       中           中   
..       ...                     ...  ...    ...     ...         ...   
95        71 2024-07-29 08:05:48.708    高      高       中           中   
96       974 2024-05-06 08:05:48.708    高      低       中           低   
97        67 2023-11-22 08:05:48.708    中      低     非常低           低   
98       415 2024-03-21 08:05:48.708    中      中       中           低   
99       976 2024-07-19 08:05:48.709    中      中       中           中   

   collisionNo bulletCollision bulletCollisionOnLiving remainHP  ...  \
0            中               低                       中        中

# Get Prediction

In [14]:
esem_factor_df = pd.read_excel('data/esem_factor.xlsx')
feature_esem_df = pd.read_excel('data/feature_esem.xlsx')

esem_factor_df['Questions'] = esem_factor_df['Questions'].astype(str)

In [16]:
import pandas as pd
import numpy as np

# Define the weights corresponding to the categories
weights = {
    '非常低': 0.25,
    '低': 0.5,
    '中': 1,
    '高': 1.5,
    '非常高': 1.75
}

# Calculate player score
def calculate_player_score(row):
    score = {'playerId': row['playerId'], '外向性': 0, '親和性': 0, '盡責性': 0, '情緒不穩定性': 0, '開放性': 0}

    for feature in feature_esem_df['Feature']:
        category = row[feature]
        weight = weights[category]

        questions = feature_esem_df.loc[feature_esem_df['Feature'] == feature, 'Questions'].values[0].split(',')

        for question in questions:
            question_factors = esem_factor_df.loc[esem_factor_df['Questions'] == question]
            
            for trait in score.keys():
                if trait == 'playerId':
                    continue
                factor_score = question_factors[trait].values[0]
                weighted_score = factor_score * weight
                score[trait] += weighted_score
    
    return score

# Calculate all player's score
def calculate_all_scores(df):
    scores = []
    for _, row in df.iterrows():
        score = calculate_player_score(row)
        scores.append(score)
    return pd.DataFrame(scores)

# Calculate median score for normalize
def calculate_median_score(df):
    median_scores = {}
    for trait in ['外向性', '親和性', '盡責性', '情緒不穩定性', '開放性']:
        median_scores[trait] = df[trait].median()
    return median_scores

# Calculate normalize score
def normalize_score(player_score, median_score):
    normalized_score = {'playerId': player_score['playerId']}
    for trait in player_score:
        if trait == 'playerId':
            continue
            
        if median_score[trait] != 0:
            normalized_score[trait] = player_score[trait] / median_score[trait]
        else:
            normalized_score[trait] = 0  # Avoid division by zero
    return normalized_score

df_scores = calculate_all_scores(df_categorized)
median_score = calculate_median_score(df_scores)

# Calculate score for specific player
player_row = df_categorized.loc[df_categorized['playerId'] == player_id]

if not player_row.empty:
    player_score = calculate_player_score(player_row.iloc[0])
    normalized_score = normalize_score(player_score, median_score)
    print(f"Normalized scores for player {player_id}:\n{normalized_score}")
else:
    print(f"Player with ID {player_id} not found.")
    
df_player = pd.DataFrame([normalized_score])

Normalized scores for player 969:
{'playerId': 969, '外向性': 1.024469160768453, '親和性': 1.029246607393542, '盡責性': 1.6454657365613468, '情緒不穩定性': 0.7382239382239387, '開放性': 0.9617933404237914}


# Update MySQL

In [18]:
import pandas as pd
from sqlalchemy import create_engine, text, Table, MetaData
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime

# Define column mapping
column_mapping = {
    'playerId': 'player_id',
    '外向性': 'extraversion',
    '親和性': 'agreeableness',
    '盡責性': 'conscientiousness',
    '情緒不穩定性': 'neuroticism',
    '開放性': 'openness'
}

# Rename columns in DataFrame
df_player.rename(columns=column_mapping, inplace=True)

# Add reportTime column to the DataFrame with current timestamp
df_player['reportTime'] = datetime.now()

# Database connection parameters
username = 'Admin'
password = '1234'
host = 'localhost'
port = '3306'
database = 'demo'

# Create a connection to the MySQL database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# Create a metadata object
metadata = MetaData()

# Reflect the table from the database
metadata.reflect(bind=engine)
table_name = 'bfi_result'
table = Table(table_name, metadata, autoload_with=engine)

# Insert or update rows
with engine.connect() as connection:
    with connection.begin() as transaction:  # Begin a transaction
        for index, row in df_player.iterrows():
            try:
                player_id = int(row['player_id'])
                # Check if the record exists
                existing = connection.execute(
                    text("SELECT COUNT(*) FROM bfi_result WHERE player_id = :player_id"),
                    {'player_id': player_id}
                ).scalar()

                if existing > 0:
                    # Update existing record
                    update_stmt = table.update().where(table.c.player_id == player_id).values(row.to_dict())
                    connection.execute(update_stmt)
                    print("Update")
                else:
                    # Insert new record
                    insert_stmt = table.insert().values(row.to_dict())
                    connection.execute(insert_stmt)
                    print("Insert")

            except SQLAlchemyError as e:
                print(f"Error occurred: {e}")
                transaction.rollback()  # Rollback the transaction if there's an error

        transaction.commit()  # Commit the transaction

Insert


# Plot Graph

In [20]:
import matplotlib.pyplot as plt
import numpy as np

# Define font
plt.rcParams['font.family'] = 'DejaVu Sans'

# Define features
features = ['extraversion', 'agreeableness', 'conscientiousness', 'neuroticism', 'openness']
features_show = ['E', 'A', 'C', 'N', 'O']

# Draw pentagon for player
row = df_player.iloc[0]

fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(polar=True))
angles = np.linspace(0, 2 * np.pi, len(features), endpoint=False).tolist()
angles += angles[:1]

values = row[features].tolist()
values += values[:1]

# Draw pentagon
ax.plot(angles, values, label=f'Player {int(row["player_id"])}')
ax.fill(angles, values, alpha=0.25)

# Set labels
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(features_show)

# Add label and save plot
plt.title(f'Player {int(row["player_id"])} Big Five Personality Scores')
plt.savefig(f'C:/xampp/htdocs/DEMO/reports/player_{int(row["player_id"])}_scores.png', bbox_inches='tight', dpi=150) 
plt.close()  # Close plot

# Produce Analysis Report

In [22]:
import os
from docx2pdf import convert

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches, Pt

# Get the report format and image directory
format_path = 'data/AnalysisReportFormat.docx'
image_directory = "C:/xampp/htdocs/DEMO/reports/"
 
# Make report for player
if not player_row.empty:
    player_bfi_row = df_player[df_player['player_id'] == player_id]
    
    if not player_bfi_row.empty:
        bfi_row = player_bfi_row.iloc[0]
        bfi_categories = {
            '{BFI_E}': round(bfi_row['extraversion'], 4),
            '{BFI_A}': round(bfi_row['agreeableness'], 4),
            '{BFI_C}': round(bfi_row['conscientiousness'], 4),
            '{BFI_N}': round(bfi_row['neuroticism'], 4),
            '{BFI_O}': round(bfi_row['openness'], 4),
            '{reportTime}':bfi_row['reportTime']
        }
        
    row = player_row.iloc[0]
    categories = {f'{{{col}}}': row[col] for col in df_categorized.columns if col != 'playerId'}

    # Get Player_name by player_id
    player_info_row = df_playerData[df_playerData['Player_id'] == player_id]

    if not player_info_row.empty:
        player_name = player_info_row.iloc[0]['Player_name']
    else:
        player_name = 'Unknown'

    # Get report format
    doc = Document(format_path)

    # Define placeholders
    placeholders = {
        '{player_name}': player_name,
        '{OCEAN_chart}': '',
        **categories,
        **bfi_categories
    }

    # Place into report
    for paragraph in doc.paragraphs:
        for placeholder, value in placeholders.items():
            if placeholder in paragraph.text:
                if placeholder == '{OCEAN_chart}':
                    chart_path = f'{image_directory}player_{row["playerId"]}_scores.png'
                    if os.path.exists(chart_path):
                        paragraph.clear() # Clear existing paragraph
                        run = paragraph.add_run()
                        run.add_picture(chart_path, width=Inches(3.25))
                        
                        # Center the image in the paragraph
                        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                    else:
                        print(f"Error: {chart_path} does not exist.")
                else:
                    paragraph.text = paragraph.text.replace(placeholder, str(value))
                    for run in paragraph.runs:
                        run.font.size = Pt(14)

    # Replace placeholders in tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for placeholder, value in placeholders.items():
                    if placeholder in cell.text:
                        cell.text = cell.text.replace(placeholder, str(value))
                # Center align the content of the cell
                for paragraph in cell.paragraphs:
                    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                    for run in paragraph.runs:
                        run.font.size = Pt(14)

    # Define paths
    docx_path = f'C:/xampp/htdocs/DEMO/reports/analysis_report_{player_id}.docx'
    pdf_path = f'C:/xampp/htdocs/DEMO/reports/analysis_report_{player_id}.pdf'

    # Save the .docx file
    doc.save(docx_path)

    # Convert .docx to PDF
    convert(docx_path, pdf_path)

    print(f'Report for player {player_name} (ID: {player_id}) saved as {pdf_path}')

  0%|          | 0/1 [00:00<?, ?it/s]

Report for player Player_969 (ID: 969) saved as C:/xampp/htdocs/DEMO/reports/analysis_report_Player_969.pdf
